<a href="https://colab.research.google.com/github/tomatencode/Mausefalle/blob/main/mausefalle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mausefalle**

In [12]:
from math import *
import scipy
import matplotlib.pyplot as plt
import numpy as np

In [19]:
u = 50 #[-] Verhältnis von Drehmoment der Feder zu Drehmoment der Achse
rr = 0.1 #[m] radius des antriebsreifen
m = 0.1 #[kg] masse des autos

friction = True
max_sim_length = 60 #[s] maximum simulation length

tolleranz = 0.05 # erlaubte abweichung des energie erhaltungs Satzes

defenirt eigenschaften für auto und simulation

$f(x)$

$\frac{1}{2}$

$\frac{df}{dx}$

$\int_0^x f(y)dy$



In [8]:
def find_Fr(v):

    return v / 20 # Wiederstand in [N] (Schätzwert)

In [15]:
def find_Ff(phi):

    if phi < pi:
        return 2 # Kraft der Feder in [Nm] (Schätzwert)
    else:
        return 0 # Kraft der Feder in [Nm]

Benutzerdefinirte Funktionen zur berechnung von Federkraft und Reibung.

In [5]:
def find_phi(x,rr,u):

    ur = 2 * pi * rr # ur[m] = 2pi[-] * rr[m]
    phi = x / ( ur * u ) # phi[-] = x[m] / (ur[m] * u[-])
    phi_rad = 2 * pi * phi # phi_rad[rad] = 2pi[-] * phi[-]

    return phi_rad # [rad]

find_phi findet den Winkel der Mausefalle in radien.
Dafür berechnet sie den umfang des Rades $ur$ mit dem radius $rr$:

$ur = 2π \cdot rr$

dann berechnet sie die umdrehungen der falle $φ$ mit der zurückgelegten Strecke $x$ und der Getriebeübersetztung $u$:

$φ = \frac{x}{ur \cdot u}$

die umrehungen weden dann in Radien umgerechnet:

$φ_{rad} = 2π \cdot φ$

In [6]:
def find_Fa(rr,u,phi):

    Ma = find_Ff(phi) # Ma[Nm] = Ff(phi[rad])[Nm]
    Me = Ma / u # Me[Nm] = Ma / ü[-]
    Fa = Me / rr # Fa[Nm] = Me[Nm] / rr[m]

    return Fa # [N]

find_Fa findet die beschleunigungskraft des Autos $F_a$.
dafur berechnet es das Drehmoment der mausefalle $M_a$ mit:

$M_a = F_f(φ)$

dann findet sie das Drehmoment der Achse $M_e$ mit der Getriebeübersetztung $u$:

$M_e = \frac{M_a}{u}$

anschließend renchnet die Funktion $M_e$ in $F_a$ mit dem Radius der Reifen $rr$ um:

$F_a = \frac{M_e}{rr}$

In [9]:
def f(t, y):

    [v,x] = y

    return [ (find_Fa(rr,u,find_phi(x,rr,u)) - find_Fr(v))/m, v]


In [14]:
def fnofric(t, y):

    [v,x] = y

    return [ find_Fa(rr,u,find_phi(x,rr,u))/m, v]

$f(t, v, x)$ ist die differenzialgleichung mit der position und Geschwindigkeit des autos berechnet werden.

$\frac{dv}{dt} = F_a - F_r$

$\frac{dx}{dt} = v$

$f_{no fric}(t, v,x)$ berechnet das selbe bei vernachlässigung der Reibung.

In [10]:
def solve_ivp(max_sim_length,friction):

    if friction == True:
        solution = scipy.integrate.solve_ivp(f, [0, max_sim_length], [0,0], method = "Radau", dense_output=True)
    else:
        solution = scipy.integrate.solve_ivp(fnofric, [0, max_sim_length], [0,0], method = "Radau", dense_output=True)
    return solution

Löst die differenzialgleichung mit oder ohne reidung ($\text{fric}$) für eine bestimmte zeit ($\text{max sim length}$) mit scipy.

In [11]:
def find_simulation_lenght(max_lenght,sulution,rr,u,v_min):

    for s in range(0,max_lenght):
        x = sulution.sol(s)[1]
        v = sulution.sol(s)[0]
        phi = find_phi(x,rr,u)
        if phi > pi and v < v_min:
            return s

    return max_lenght

findet passende simulationslänge.

Returnd wenn $v < \text{v_min}$ und $φ > π.$

In [17]:
def find_Wmf(M, phi):

    Wmf = M * phi # e[J] = M[Nm] * phi[rad]

    return Wmf #[J]

um zu testen ob der Energieerhaltungssatzt verletzt wurde findet $W_{mf}(M, φ)$ die arbeit die von der mausefalle verrichtet wurde mit dem Drehmoment $M$ und dem winkel $φ$ mit der Formel:

$W_{mf} = M \cdot φ$

(diese formel funktionirt nur für eine statische federkraft)

In [18]:
def find_Wacc(m,ve,va):

    Wacc = (1/2) * m * ve**2 - (1/2) * m * va**2 # Wacc[J] = 1/2m[kg]ve[m/s]² - 1/2m[kg]va[m/s]²

    return Wacc #[J]

um zu testen ob der Energieerhaltungssatzt verletzt wurde findet $W_{acc}(m, v_e, v_a)$ die Beschleunigungsarbeit des autos mit der anfangsgeschwindigkeit $v_a$, der Endgeschwindigkeit $v_e$ und der Masse $m$, diese wird berechnet mit der Formel:

$W_{acc} = \frac{1}{2}m \cdot v_e - \frac{1}{2}m \cdot v_a$

In [20]:
def test(do_print):
    solution = solve_ivp(max_sim_length,False)
    ve = solution.sol(max_sim_length)[0]
    va = solution.sol(0)[0]

    Wmf = find_Wmf(2,pi)
    Wacc = find_Wacc(m,ve,va)
    abweichung = abs(Wmf - Wacc)

    print("Emf = "+str(Wmf) + " J und Eacc = " + str(Wacc)+ " J")

    if do_print:
        if abweichung < tolleranz:
            print("kein bug! abweichung: " + str(abweichung) + " J")
        else:
            print("abweichung über tolleranz! abweichung: " + str(abweichung) + " J")
    return abweichung

prüft ob der Energieerhaltungssatzt verletzt wurde indem sie die arbeit der feder und die der beschleunigung vergleicht.

In [ ]:
solution = solve_ivp(max_sim_length,friction) # lösung der differenzialgleichung [-]

s = find_simulation_lenght(max_sim_length,solution,rr,u,0.1) # passende länge der simulation [s]

t = np.linspace(0, s, 300) # vektor von 300 zeitpunkten von 0 bis s [s]

plt.plot(t, solution.sol(t)[0].T,label='Speed[m/s]',color="green")
plt.plot(t, solution.sol(t)[1].T,label='position[m]',color=(1,0.55,0))
plt.plot(t, find_phi(solution.sol(t)[1],rr,u).T,label='Winkel Mausefalle[rad]',color=(0.58,0,0.82))
if friction == True:
    plt.plot(t, (Fr(solution.sol(t)[0])/m).T,label='Reibung[m/s²]',color="red")

plt.xlabel('t')
plt.legend()
plt.title('Auto')

plt.show()
test(True) # testet ob der energieerhaltungs satz verletzt wurde

plotet Geschwindigkeit Position Reibung und Winkel der Mausefalle.

In [ ]:
from kivy.app import App
from kivy.properties import Clock
from kivy.uix.relativelayout import RelativeLayout
from kivy.graphics.context_instructions import Color
from kivy.graphics.vertex_instructions import Ellipse, Triangle, Rectangle
from kivy.properties import StringProperty
from main import solve_ivp, find_Fa, find_phi, Fr, rr, u, m, friction, max_sim_length
from energiesummentest import test


class MainWidget(RelativeLayout):

    size_car = 0
    time_running = 0
    track_start = .1
    track_length = .8
    car_x = 0

    arrow_speed_length = 0
    arrow_acceleration_length = 0
    arrow_friction_length = 0


    sim_speed = 1

    sim_play = False
    solution = solve_ivp(max_sim_length,friction)
    car_a_label = StringProperty("a: 0")
    car_v_label = StringProperty("v: 0")
    car_x_label = StringProperty("x: 0")
    car_t_label = StringProperty("t: 0")


    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.innit_speed_arrow()
        self.innit_acceleration_arrow()
        self.innit_friction_arrow()
        self.innit_car()

        Clock.schedule_interval(self.update,1.0/60.0) # ruft update() 60 mal pro sekunde

    def innit_car(self):
        """
        definirt die formen des autos
        """
        with self.canvas.before:
            Color(1,1,1)
            self.car = Ellipse()
        with self.canvas:
            Color(.7,.7,.7)
            self.arrow_base_circle_1 = Ellipse()
            self.arrow_base_circle_2 = Ellipse()

    def innit_speed_arrow(self):
        """
        definirt die formen des geschwindigkeits pfeil
        """
        with self.canvas:
            Color(0,0.5,0)
            self.speed_arrow_base = Rectangle()
        with self.canvas.after:
            Color(0,0.6,0)
            self.speed_arrow_tip = Triangle()

    def innit_acceleration_arrow(self):
        """
        definirt die formen des beschleunigungs pfeil
        """
        with self.canvas:
            Color(0,0,0.5)
            self.acceleration_arrow_base = Rectangle()
        with self.canvas.after:
            Color(0,0,0.6)
            self.acceleration_arrow_tip = Triangle()

    def innit_friction_arrow(self):
        """
        definirt die formen des reibungs pfeil
        """
        with self.canvas:
            Color(0.6,0,0)
            self.friction_arrow_base = Rectangle()
        with self.canvas.after:
            Color(0.7,0,0)
            self.friction_arrow_tip = Triangle()

    def start_stop(self,obj):
        """
        startet und stopt die simulation
        """
        if obj.state == "down":
            self.sim_play = True
            obj.text = "stop"
        if obj.state == "normal":
            self.sim_play = False
            obj.text = "start"

    def restart(self):
        """
        starttet die simulation neu
        """
        self.time_running = 0

    def speed_change(self,obj):
        self.sim_speed = obj.value

    def draw_triangle(self,extra_y,length,direction):
        """
        berechnet die koordinaten der punkte eines pfeils
        """
        arrow_pos_x = self.car_x+self.size_car*0.5
        arrow_y = self.size_car*0.5
        arrow_base_pos = arrow_pos_x,arrow_y-self.size_car*0.08+extra_y
        arrow_base_size = length,self.size_car*0.16
        point1 = (arrow_pos_x+length,arrow_y-self.height/75+extra_y)
        point2 = (arrow_pos_x+length,arrow_y+self.height/75+extra_y)
        point3 = (arrow_pos_x+length+(self.width/60)*direction,arrow_y+extra_y)
        return (arrow_base_pos,arrow_base_size,(*point1, *point2, *point3))

    def update_speed_arrow(self):
        self.arrow_speed_length = self.speed*10*(self.width/800)
        base_pos,base_size,points = self.draw_triangle(self.size_car*-0.3,self.arrow_speed_length,1)
        self.speed_arrow_base.pos = base_pos
        self.speed_arrow_base.size = base_size
        self.speed_arrow_tip.points = [*points]

    def update_acceleration_arrow(self):
        self.arrow_acceleration_length = self.acceleration*10*(self.width/800)
        base_pos,base_size,points = self.draw_triangle(self.size_car*0.3,self.arrow_acceleration_length,1)
        self.acceleration_arrow_base.pos = base_pos
        self.acceleration_arrow_base.size = base_size
        self.acceleration_arrow_tip.points = [*points]

    def update_friction_arrow(self):
        if friction == True:
            self.arrow_friction_length = self.friction*10*(self.width/800)
        else:
            self.arrow_friction_length = 0
        base_pos,base_size,points = self.draw_triangle(self.size_car*0.3,-self.arrow_friction_length,-1)
        self.friction_arrow_base.pos = base_pos
        self.friction_arrow_base.size = base_size
        self.friction_arrow_tip.points = [*points]

    def update_car(self):
        self.size_car = self.width/20
        self.max_distance = self.solution.sol(max_sim_length)[1]
        self.distance_percent = self.distance / self.max_distance
        self.car_x = (self.track_start*self.width) + (self.track_length*self.width) * self.distance_percent
        self.car.size = (self.size_car,self.size_car)
        self.car.pos = (self.car_x,0)
        self.arrow_base_circle_1.size = (self.size_car*0.3,self.size_car*0.3)
        self.arrow_base_circle_1.pos = (self.car_x+self.size_car*0.35,self.size_car*0.05)
        self.arrow_base_circle_2.size = (self.size_car*0.3,self.size_car*0.3)
        self.arrow_base_circle_2.pos = (self.car_x+self.size_car*0.35,self.size_car*0.65)

    def update_lables(self):
        self.car_a_label = "a: " + str(round(self.acceleration-self.friction,1)) + "m/s²"
        self.car_v_label = "v: " + str(round(self.speed,1)) + "m/s"
        self.car_x_label = "x: " + str(round(self.distance,1)) + "m"
        self.car_t_label = "t: " + str(round(self.time_running,1)) + "s"

    def update(self,dt):
        """
        updatet alle formen
        """

        if self.sim_play:
            self.time_running += dt*self.sim_speed

        self.distance = self.solution.sol(self.time_running)[1]
        self.speed = self.solution.sol(self.time_running)[0]
        self.acceleration = find_Fa(rr,u,find_phi(self.distance,rr,u))/m
        self.friction = (Fr(self.speed))/m

        self.update_speed_arrow()
        self.update_acceleration_arrow()
        self.update_friction_arrow()
        self.update_lables()
        self.update_car()

class AutoAnimationApp(App):
    def build(self):
        self.load_string("""
        MainWidget:

            BoxLayout:
                pos_hint: {"x":.025,"y":.775}
                size_hint: (.35,.2)
                orientation: 'vertical'
                BoxLayout:
                    ToggleButton:
                        size_hint: (.9,.9)
                        on_press: root.start_stop(self)
                        text: "start"
                        font_size: str(root.width/50)+"dp"
                    Button:
                        size_hint: (.9,.9)
                        on_press: root.restart()
                        text: "restart"
                        font_size: str(root.width/50)+"dp"
                Label:
                    text: "sim speed: " + str(round(slider.value,1))
                    font_size: str(root.width/50)+"dp"
                Slider:
                    id: slider
                    min: .1
                    max: 2
                    value: 1
                    on_value: root.speed_change(self)

            BoxLayout:
                pos_hint: {"x":.8,"y":.65}
                size_hint: (.15,.3)
                orientation: 'vertical'
                Label:
                    text: root.car_a_label
                    font_size: str(root.width/50)+"dp"
                Label:
                    text: root.car_v_label
                    font_size: str(root.width/50)+"dp"
                Label:
                    text: root.car_x_label
                    font_size: str(root.width/50)+"dp"
                Label:
                    text: root.car_t_label
                    font_size: str(root.width/50)+"dp"

            BoxLayout:
                pos_hint: {"x":.45,"y":.9}
                size_hint: (.3,.05)
                orientation: 'vertical'
                BoxLayout:
                    spacing: 10
                    size_hint: (1,.07)
                    RelativeLayout:
                        canvas:
                            Color:
                                rgb: 0, .6, 0
                            Rectangle:
                                size: self.size
                    RelativeLayout:
                        canvas:
                            Color:
                                rgb: 0, 0, .6
                            Rectangle:
                                size: self.size
                    RelativeLayout:
                        canvas:
                            Color:
                                rgb: .7, 0, 0
                            Rectangle:
                                size: self.size
                BoxLayout:
                    Label:
                        text: "Speed"
                        font_size: str(root.width/50)+"dp"
                    Label:
                        text: "Acceleration"
                        font_size: str(root.width/50)+"dp"
                    Label:
                        text: "Friction"
                        font_size: str(root.width/50)+"dp"
        """)

AutoAnimationApp().run() # startet die app
test(True) # testet ob der energie erhaltungssatz verletzt wurde

Animation